In [ ]:
!freqtrade backtesting --strategy FreqAIDynamicClassifierStrategy --userdir /allah/stuff/freq/project_1/user_data --config /allah/stuff/freq/project_1/user_data/config_freqai.json --timerange 20250110-20250111 --datadir /allah/freqtrade/user_data/data/binance --cache none --starting-balance 10000 --eps --export signals --fee 0 --freqaimodel PyTorchMLPClassifier

In [8]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Any
from pathlib import Path

# Configuration
CONFIG = {
    'backtest_dir': '/allah/stuff/freq/project_1/user_data/backtest_results/',
    'output_dir': '/allah/data/parquet/',
    'strategy_name': 'TrendReversalLabelingStrategy'
}


In [ ]:
def load_backtest_data() -> Dict[str, Any]:
    """Load the latest backtest results"""
    try:
        last_result_path = Path(CONFIG['backtest_dir']) / '.last_result.json'
        with open(last_result_path, 'r') as f:
            last_result = json.load(f)
        
        backtest_path = Path(CONFIG['backtest_dir']) / last_result['latest_backtest']
        with open(backtest_path, 'r') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading data: {e}")
        return {}

def process_trades(backtest_data: Dict[str, Any]) -> pd.DataFrame:
    """Process backtest trades data"""
    trades_df = pd.DataFrame(
        backtest_data['strategy'][CONFIG['strategy_name']]['trades']
    )
    
    # Select and process columns
    cols = ['pair', 'open_date', 'close_date', 'profit_ratio', 'is_short', 'stop_loss_ratio']
    df = trades_df[cols].copy()
    
    # Convert dates
    df['open_date'] = pd.to_datetime(df['open_date'])
    df['close_date'] = pd.to_datetime(df['close_date'])
    
    # Add profitability classification
    df['profitability'] = df['profit_ratio'].apply(
        lambda x: 'WIN' if x > 0 else 'LOSE' if x < 0 else 'NEUTRAL'
    )
    
    return df


def analyze_monthly_metrics(df: pd.DataFrame) -> None:
    """Detailed monthly analysis of various trading metrics"""
    
    metrics = {
        'Profit Ratio': {
            'all': df.set_index('open_date')['profit_ratio'].resample('ME').mean(),
            'profitable': df[df['profit_ratio'] > 0].set_index('open_date')['profit_ratio'].resample('ME').mean()
        },
        'Trade Duration': {
            'all': (df['close_date'] - df['open_date']).dt.total_seconds() / 3600,  # hours
            'profitable': (df[df['profit_ratio'] > 0]['close_date'] - 
                         df[df['profit_ratio'] > 0]['open_date']).dt.total_seconds() / 3600
        },
        'Stop Loss': {
            'ratio': df.groupby(pd.Grouper(key='open_date', freq='ME'))['stop_loss_ratio'].mean(),
            'hits': df.groupby(pd.Grouper(key='open_date', freq='ME'))['stop_loss_ratio'].count()
        },
        'Volume': {
            'by_pair': df.groupby([pd.Grouper(key='open_date', freq='ME'), 'pair']).size().unstack(fill_value=0)
        }
    }
    
    # Create figure and axes
    fig, axes = plt.subplots(5, 2, figsize=(15, 25))
    
    # Profit Analysis
    metrics['Profit Ratio']['all'].plot(ax=axes[0,0])
    axes[0,0].set_title('Monthly Avg Profit - All Trades')
    axes[0,0].grid(True)
    
    metrics['Profit Ratio']['profitable'].plot(ax=axes[0,1], color='green')
    axes[0,1].set_title('Monthly Avg Profit - Profitable Only')
    axes[0,1].grid(True)
    
    # Duration Analysis
    axes[1,0].hist(metrics['Trade Duration']['all'], bins=50)
    axes[1,0].set_title('Trade Duration Dist - All')
    axes[1,0].set_xlabel('Hours')
    axes[1,0].grid(True)
    
    axes[1,1].hist(metrics['Trade Duration']['profitable'], bins=50, color='green')
    axes[1,1].set_title('Trade Duration Dist - Profitable')
    axes[1,1].set_xlabel('Hours')
    axes[1,1].grid(True)
    
    # Stop Loss Analysis
    metrics['Stop Loss']['ratio'].plot(ax=axes[2,0])
    axes[2,0].set_title('Monthly Avg Stop Loss Ratio')
    axes[2,0].grid(True)
    
    metrics['Stop Loss']['hits'].plot(ax=axes[2,1])
    axes[2,1].set_title('Monthly Stop Loss Hits')
    axes[2,1].grid(True)
    
    # Volume Analysis
    metrics['Volume']['by_pair'].plot(ax=axes[3,0])
    axes[3,0].set_title('Monthly Trade Volume by Pair')
    axes[3,0].grid(True)
    
    metrics['Volume']['by_pair'].mean().plot(kind='bar', ax=axes[3,1])
    axes[3,1].set_title('Avg Monthly Volume by Pair')
    axes[3,1].grid(True)
    
    # Win Rate Analysis
    monthly_win_rate = (df.set_index('open_date')
                       .groupby(pd.Grouper(freq='ME'))['profitability']
                       .apply(lambda x: (x == 'WIN').mean()))
    monthly_win_rate.plot(ax=axes[4,0])
    axes[4,0].set_title('Monthly Win Rate')
    axes[4,0].grid(True)
    
    df['profitability'].value_counts(normalize=True).plot(kind='bar', ax=axes[4,1])
    axes[4,1].set_title('Overall Win/Loss Distribution')
    axes[4,1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\nSummary Statistics:")
    for metric, data in metrics.items():
        print(f"\n{metric}:")
        for name, values in data.items():
            if isinstance(values, pd.Series):
                print(f"\n{name.title()}:")
                print(values.describe())

try:
    analyze_monthly_metrics(df)
except Exception as e:
    print(f"Error during analysis: {e}")

In [5]:
def analyze_profitability(df: pd.DataFrame) -> None:
    """Analyze and visualize profitability metrics including profitable trades only"""
    
    plt.figure(figsize=(15, 12))
    
    # All trades monthly profit
    plt.subplot(3, 1, 1)
    monthly_profit = df.set_index('open_date')['profit_ratio'].resample('ME').mean()
    monthly_profit.plot(kind='line', marker='o')
    plt.title('Monthly Average Profit Ratio (All Trades)')
    plt.grid(True)
    
    # Profitable trades only monthly profit 
    plt.subplot(3, 1, 2)
    profitable_df = df[df['profit_ratio'] > 0]
    monthly_profit_positive = profitable_df.set_index('open_date')['profit_ratio'].resample('ME').mean()
    monthly_profit_positive.plot(kind='line', marker='o', color='green')
    plt.title('Monthly Average Profit Ratio (Profitable Trades Only)')
    plt.grid(True)
    
    # Win/Loss distribution
    plt.subplot(3, 1, 3)
    df['profitability'].value_counts().plot(kind='bar')
    plt.title('Win/Loss Distribution')
    
    plt.tight_layout()
    plt.show()
    
    # Print stats
    print("\nMonthly Profit Statistics:")
    print("\nAll Trades:")
    print(monthly_profit.describe())
    print("\nProfitable Trades Only:")
    print(monthly_profit_positive.describe())

def analyze_pairs(df: pd.DataFrame) -> None:
    """Analyze performance by trading pair"""
    pair_stats = df.groupby('pair').agg({
        'profit_ratio': ['mean', 'count'],
        'profitability': lambda x: (x == 'WIN').mean()
    }).round(4)
    
    print("\nPair Statistics:")
    print(pair_stats)


In [ ]:
if __name__ == "__main__":
    # Load and process data
    backtest_data = load_backtest_data()
    df = process_trades(backtest_data)
    
    # Remove NEUTRAL results
    df = df[df['profitability'] != 'NEUTRAL']
    
    # Run analysis
    analyze_profitability(df)
    analyze_pairs(df)
    
    # Export processed data
    output_path = Path(CONFIG['output_dir']) / 'filtered_df.parquet'
    df.to_parquet(output_path)
    print(f"\nData exported to {output_path}")
